In [1]:
%cd ../..
%load_ext autoreload
%autoreload 2
import datajoint as dj
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20})
from pipeline import lab, get_schema_name, experiment, foraging_model, ephys, foraging_analysis, histology, ccf
from pipeline.plot import unit_psth
from pipeline.plot.foraging_model_plot import plot_session_model_comparison, plot_session_fitted_choice

/home/jupyter/map-ephys


[2022-11-17 22:40:08,162][INFO]: Connecting map-superuser@datajoint.mesoscale-activity-map.org:3306
[2022-11-17 22:40:08,212][INFO]: Connected map-superuser@datajoint.mesoscale-activity-map.org:3306


Total foraging units

In [2]:
q_unit_foraging = ephys.Unit & foraging_analysis.SessionTaskProtocol
total = len(q_unit_foraging)
total

40845

In [6]:
qc_criteria = {
    'before': 'presence_ratio > 0.9 '
              'AND amplitude_cutoff < 0.1 '
              'AND isi_violation < 0.5 '
              'AND unit_amp > 70',
    'minimal': 'unit_amp > 70 '
               'AND avg_firing_rate > 0.1 '
               'AND presence_ratio > 0.9 '
               'AND isi_violation < 0.1 '
               'AND amplitude_cutoff < 0.15',
    'Medulla': 'unit_amp > 150 '
               'AND avg_firing_rate > 0.2 '
               'AND presence_ratio > 0.9 '
               'AND isi_violation < 10 '
               'AND amplitude_cutoff < 0.15',
    'ALM': 'unit_amp > 100 '
           'AND avg_firing_rate > 0.2 '
           'AND presence_ratio > 0.95 '
           'AND isi_violation < 0.1 '
           'AND amplitude_cutoff < 0.1',
    'Midbrain': 'unit_amp > 100 '
                'AND avg_firing_rate > 0.1 '
                'AND presence_ratio > 0.9 '
                'AND isi_violation < 1 '
                'AND amplitude_cutoff < 0.08',
    'Thalamus': 'unit_amp > 90 '
                'AND avg_firing_rate > 0.1 '
                'AND presence_ratio > 0.9 '
                'AND isi_violation < 0.05 '
                'AND amplitude_cutoff < 0.08',
    'Striatum': 'unit_amp > 70 '
                'AND avg_firing_rate > 0.1 '
                'AND presence_ratio > 0.9 '
                'AND isi_violation < 0.5 '
                'AND amplitude_cutoff < 0.1'
}

In [7]:
for qc in qc_criteria:
    qced = q_unit_foraging * ephys.ClusterMetric * ephys.UnitStat & qc_criteria[qc]
    print(f'{qc}: {len(qced)}, {len(qced)/total:.2%}')

before: 11682, 28.60%
minimal: 9142, 22.38%
Medulla: 4601, 11.26%
ALM: 5900, 14.44%
Midbrain: 8099, 19.83%
Thalamus: 6099, 14.93%
Striatum: 11620, 28.45%


In [14]:
(ephys.ProbeInsertion * experiment.Session & 'username="hh"' & histology.ElectrodeCCFPosition())

subject_id institution 6 digit animal ID,session session number,insertion_number,probe unique identifier for this model of probe (e.g. part number),probe_type e.g. neuropixels_1.0,electrode_config_name user friendly name,session_date,session_time,username,rig
473360,49,1,18005123491,neuropixels 1.0 - 3B,1-384,2021-08-12,16:52:05,HH,Ephys-Han
473360,49,2,18005108622,neuropixels 1.0 - 3B,1-384,2021-08-12,16:52:05,HH,Ephys-Han
473360,50,1,18005123491,neuropixels 1.0 - 3B,1-384,2021-08-13,16:33:03,HH,Ephys-Han
473360,50,2,18005108622,neuropixels 1.0 - 3B,1-384,2021-08-13,16:33:03,HH,Ephys-Han
473360,51,1,18005108622,neuropixels 1.0 - 3B,1-384,2021-08-14,18:08:26,HH,Ephys-Han
473360,51,2,18005123491,neuropixels 1.0 - 3B,1-384,2021-08-14,18:08:26,HH,Ephys-Han
473360,51,3,19122505354,neuropixels 2.0 - SS,1-384,2021-08-14,18:08:26,HH,Ephys-Han
473360,52,1,18005108622,neuropixels 1.0 - 3B,1-384,2021-08-15,18:19:35,HH,Ephys-Han
473360,52,2,18005123491,neuropixels 1.0 - 3B,1-384,2021-08-15,18:19:35,HH,Ephys-Han
473360,52,3,18005117122,neuropixels 1.0 - 3B,1-384,2021-08-15,18:19:35,HH,Ephys-Han


In [6]:
from pipeline import foraging_analysis

foraging_ephys_no_photostim = (experiment.Session * lab.WaterRestriction.proj('water_restriction_number') 
                                & (foraging_analysis.SessionTaskProtocol & 'session_task_protocol = 100')
                                & ephys.ProbeInsertion - experiment.PhotostimForagingTrial)
session_qc = 'trial_number > 300 AND foraging_efficiency > 0.75'  # An arbitrary QC for trial number and performance
foraging_sess_qc = (foraging_ephys_no_photostim & (foraging_analysis.SessionStats.proj(trial_number='session_pure_choices_num', 
                                                         foraging_efficiency='session_foraging_eff_optimal_random_seed') 
                             & session_qc))
foraging_sess_qc

subject_id institution 6 digit animal ID,session session number,session_date,session_time,username,rig,water_restriction_number WR number
473360,49,2021-08-12,16:52:05,HH,Ephys-Han,HH08
473360,50,2021-08-13,16:33:03,HH,Ephys-Han,HH08
473360,51,2021-08-14,18:08:26,HH,Ephys-Han,HH08
473360,52,2021-08-15,18:19:35,HH,Ephys-Han,HH08
473361,47,2021-04-18,23:00:03,HH,Ephys-Han,HH09
473361,57,2021-06-09,16:28:15,HH,Ephys-Han,HH09
473361,59,2021-06-12,14:18:29,HH,Ephys-Han,HH09
473361,60,2021-06-13,20:27:10,HH,Ephys-Han,HH09
482353,33,2021-11-02,17:05:06,HH,Ephys-Han,HH13
482353,34,2021-11-03,16:49:36,HH,Ephys-Han,HH13
